In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType

In [0]:
trips = spark.table("default.staging_trips")
display(trips.limit(10))

trip_id,rideable_type,start_at,ended_at,start_station_id,end_station_id,rider_id
89E7AA6C29227EFF,classic_bike,2021-02-12T16:14:56.000+0000,2021-02-12T16:21:43.000+0000,525,660,71934
0FEFDE2603568365,classic_bike,2021-02-14T17:52:38.000+0000,2021-02-14T18:12:09.000+0000,525,16806,47854
E6159D746B2DBB91,electric_bike,2021-02-09T19:10:18.000+0000,2021-02-09T19:19:10.000+0000,KA1503000012,TA1305000029,70870
B32D3199F1C2E75B,classic_bike,2021-02-02T17:49:41.000+0000,2021-02-02T17:54:06.000+0000,637,TA1305000034,58974
83E463F23575F4BF,electric_bike,2021-02-23T15:07:23.000+0000,2021-02-23T15:22:37.000+0000,13216,TA1309000055,39608
BDAA7E3494E8D545,electric_bike,2021-02-24T15:43:33.000+0000,2021-02-24T15:49:05.000+0000,18003,KP1705001026,36267
A772742351171257,classic_bike,2021-02-01T17:47:42.000+0000,2021-02-01T17:48:33.000+0000,KP1705001026,KP1705001026,50104
295476889D9B79F8,classic_bike,2021-02-11T18:33:53.000+0000,2021-02-11T18:35:09.000+0000,18003,18003,19618
362087194BA4CC9A,classic_bike,2021-02-27T15:13:39.000+0000,2021-02-27T15:36:36.000+0000,KP1705001026,KP1705001026,16732
21630F715038CCB0,classic_bike,2021-02-20T08:59:42.000+0000,2021-02-20T09:17:04.000+0000,KP1705001026,KP1705001026,57068


In [0]:
trip_start_at = trips.select('start_at').rdd.flatMap(lambda x: x).collect()
trip_end_at = trips.select('ended_at').rdd.flatMap(lambda x: x).collect()
date = trip_start_at + trip_end_at
time_df = spark.createDataFrame(date, TimestampType())
dim_datetimes = time_df.toDF("datetime") \
                .withColumn('hour', hour('datetime')) \
                .withColumn('dayofmonth', dayofmonth('datetime')) \
                .withColumn('weekofyear', weekofyear('datetime')) \
                .withColumn('dayofweek', dayofweek('datetime')) \
                .withColumn('month', month('datetime')) \
                .withColumn('quarter', quarter('datetime')) \
                .withColumn('year', year('datetime')) \
                .distinct()
display(dim_datetimes.limit(10))

datetime,hour,dayofmonth,weekofyear,dayofweek,month,quarter,year
2021-02-02T17:49:41.000+0000,17,2,5,3,2,1,2021
2021-02-14T17:52:38.000+0000,17,14,6,1,2,1,2021
2021-02-01T17:47:42.000+0000,17,1,5,2,2,1,2021
2021-02-09T19:10:18.000+0000,19,9,6,3,2,1,2021
2021-02-23T15:07:23.000+0000,15,23,8,3,2,1,2021
2021-02-20T08:59:42.000+0000,8,20,7,7,2,1,2021
2021-02-24T15:43:33.000+0000,15,24,8,4,2,1,2021
2021-02-27T15:13:39.000+0000,15,27,8,7,2,1,2021
2021-02-12T16:14:56.000+0000,16,12,6,6,2,1,2021
2021-02-11T18:33:53.000+0000,18,11,6,5,2,1,2021


Output can only be rendered in Databricks

In [0]:
dim_datetimes.write.format("delta").mode("overwrite").saveAsTable("default.dim_dates")